In [1]:
%pip install -r requirements.txt -q

Note: you may need to restart the kernel to use updated packages.


In [4]:
from dotenv import load_dotenv
import os

load_dotenv()
api_key = os.getenv('API_KEY')
os.environ["OPENAI_API_KEY"] = api_key

In [11]:
import os
import requests
from io import BytesIO
from tempfile import NamedTemporaryFile
from langchain.document_loaders import PyPDFLoader
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA

def extract_text_from_pdf(file_path):
    """Extract text from a PDF file."""
    pdf_loader = PyPDFLoader(file_path)
    documents = pdf_loader.load()
    #os.remove(file_path)  # Remove the file if it's temporarily stored
    return documents

def setup_retrieval_chain(documents):
    """Set up the retrieval chain using documents."""
    embeddings = OpenAIEmbeddings()
    vectorstore = FAISS.from_documents(documents, embeddings)
    vectorstore.save_local("faiss_index")
    retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 5})

    retrieval_qa = RetrievalQA.from_chain_type(
        llm=ChatOpenAI(model="gpt-3.5-turbo"),
        chain_type="refine",
        retriever=retriever,
        return_source_documents=True
    )
    return retrieval_qa


def ask_question(retrieval_qa, question):
    """Ask a question and retrieve answers."""
    result = retrieval_qa.invoke({"query": question})
    print("Answer:", result['result'])

    source_docs_excerpts = [
        "Source Document: " + doc.page_content[:200] + "..." for doc in result['source_documents']
    ]
    print(source_docs_excerpts)

if __name__ == "__main__":
    pdf_file_path = '/Users/mira/VScode/chatbot/data/sample.pdf'
    os.environ["OPENAI_API_KEY"] = api_key

    documents = extract_text_from_pdf(pdf_file_path)
    retrieval_qa = setup_retrieval_chain(documents)
    question = "この人は何をしている人ですか？"
    ask_question(retrieval_qa, question)


Answer: この人はデータサイエンスに情熱を注いでおり、機械学習のモデルを作成し、データから新たな洞察を引き出すことに取り組んでいます。特に、予測モデリングとデータ分析が得意であり、常に新しい技術を学び、それを活用して課題を解決することを目指しています。プログラミングや読書が趣味で、新しい知識を吸収することが大好きです。将来もデータの世界で新たな価値を生み出すために努力していく意向です。
['Source Document: 初\nめ\nま\nし\nて！\nデー\nタ\nサ\nイ\nエ\nン\nス\nに\n情\n熱\nを\n注\nい\nで\nい\nる\n者\nで\nす。\n機\n械\n学\n習\nの\nモ\nデ\nル\nを\n作\n成\nし、\nデー\nタ\nか\nら\n新\nた\nな\n洞\n察\nを\n引\nき\n出\nす\nこ\nと\nに\n日々\n取\nり\n組\nん\nで\nい\nま\nす。\n特\nに、\n予\n測\nモ\nデ\nリ\nン\nグ\nと\nデー\nタ\n分\n析\nが\n得\n意\nで\nす。\nプ\nロ\nジェ\nク\nト\nで\nは、\n常\nに\n新\nし\nい\n技\n...']
